# **Extraer Noticias desde Yahoo Finance**

In [1]:
# Beatiful Soup
%pip install yahoo_fin pyarrow requests_html lxml_html_clean beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup 
from yahoo_fin import news 
import warnings
import pandas as pd
import concurrent.futures
import unicodedata
import json
import re 

## Obteniendo las noticias relacionadas con el IPC

In [3]:
stock_news = news.get_yf_rss("^MXX")
df = pd.DataFrame(stock_news)
df

,summary,summary_detail,id,guidislink,links,link,published,published_parsed,title,title_detail
0,While the is up nearly 4% in 2025 and isn’t f...,"{'type': 'text/html', 'language': None, 'base'...",b5625d64-fbed-3b0f-a232-17e0fc421b37,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/m/b5625d64-fbed-3b0f...,"Thu, 20 Feb 2025 19:57:00 +0000","(2025, 2, 20, 19, 57, 0, 3, 51, 0)",The S&P 500 Could Keep Climbing. These Global ...,"{'type': 'text/plain', 'language': None, 'base..."
1,* Chile's peso at four month highs * Rising...,"{'type': 'text/html', 'language': None, 'base'...",1905da4a-7bc5-3840-b797-7383b6e3f1f2,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Thu, 16 May 2024 14:55:41 +0000","(2024, 5, 16, 14, 55, 41, 3, 137, 0)",EMERGING MARKETS-Most Latam FX fall as dollar ...,"{'type': 'text/plain', 'language': None, 'base..."
2,* Petrobras slumps after its CEO to step down...,"{'type': 'text/html', 'language': None, 'base'...",887dc1b6-1ece-3d6b-bb46-e0b220e3488a,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Wed, 15 May 2024 19:40:42 +0000","(2024, 5, 15, 19, 40, 42, 2, 136, 0)","EMERGING MARKETS-Mexico, Chile lead Latam FX u...","{'type': 'text/plain', 'language': None, 'base..."
3,* Petrobras slumps after its CEO to step down...,"{'type': 'text/html', 'language': None, 'base'...",c141dfc5-ea0c-3778-843f-b503bd4966ba,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Wed, 15 May 2024 15:07:14 +0000","(2024, 5, 15, 15, 7, 14, 2, 136, 0)",EMERGING MARKETS-Most Latam FX up on softer U....,"{'type': 'text/plain', 'language': None, 'base..."
4,* Petrobras shares fall after fuel sales hurt...,"{'type': 'text/html', 'language': None, 'base'...",90b5475c-6953-384e-8d8c-0eb024d77f3c,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Tue, 14 May 2024 20:06:12 +0000","(2024, 5, 14, 20, 6, 12, 1, 135, 0)","EMERGING MARKETS-Colombia, Chile outperform br...","{'type': 'text/plain', 'language': None, 'base..."
5,* Brazil's Hapvida jumps after higher Q1 prof...,"{'type': 'text/html', 'language': None, 'base'...",7715d46c-04c7-314c-af28-df2946fd7354,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Tue, 14 May 2024 15:08:09 +0000","(2024, 5, 14, 15, 8, 9, 1, 135, 0)",EMERGING MARKETS-Most Latam FX inch up in lead...,"{'type': 'text/plain', 'language': None, 'base..."
6,* Banxico chief says rate cut a possibility a...,"{'type': 'text/html', 'language': None, 'base'...",897babf7-12ec-3526-a95a-f07d28cd9f40,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Mon, 13 May 2024 19:45:43 +0000","(2024, 5, 13, 19, 45, 43, 0, 134, 0)","EMERGING MARKETS-Latam FX, stocks flat as inve...","{'type': 'text/plain', 'language': None, 'base..."


### Obtener el contenido completo de la noticia

In [4]:
def extraer_contenido(df: pd.DataFrame) -> pd.DataFrame:
    content = []

    for url in df['link']:
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0'
            }
            response = requests.get(url, headers=headers)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                parrafo = soup.find('p', {'class': 'yf-1090901'})
                texto = parrafo.text.strip() if parrafo else 'No se encontró contenido'
            else:
                texto = f'Error {response.status_code}'

        except Exception as e:
            texto = f'Error: {str(e)}'

        content.append(texto)

    df['contenido'] = content
    return df

In [5]:
df = extraer_contenido(df)
df

,summary,summary_detail,id,guidislink,links,link,published,published_parsed,title,title_detail,contenido
0,While the is up nearly 4% in 2025 and isn’t f...,"{'type': 'text/html', 'language': None, 'base'...",b5625d64-fbed-3b0f-a232-17e0fc421b37,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/m/b5625d64-fbed-3b0f...,"Thu, 20 Feb 2025 19:57:00 +0000","(2025, 2, 20, 19, 57, 0, 3, 51, 0)",The S&P 500 Could Keep Climbing. These Global ...,"{'type': 'text/plain', 'language': None, 'base...",While the is up nearly 4% in 2025 and isn’t fa...
1,* Chile's peso at four month highs * Rising...,"{'type': 'text/html', 'language': None, 'base'...",1905da4a-7bc5-3840-b797-7383b6e3f1f2,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Thu, 16 May 2024 14:55:41 +0000","(2024, 5, 16, 14, 55, 41, 3, 137, 0)",EMERGING MARKETS-Most Latam FX fall as dollar ...,"{'type': 'text/plain', 'language': None, 'base...",* Chile's peso at four month highs * Rising ir...
2,* Petrobras slumps after its CEO to step down...,"{'type': 'text/html', 'language': None, 'base'...",887dc1b6-1ece-3d6b-bb46-e0b220e3488a,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Wed, 15 May 2024 19:40:42 +0000","(2024, 5, 15, 19, 40, 42, 2, 136, 0)","EMERGING MARKETS-Mexico, Chile lead Latam FX u...","{'type': 'text/plain', 'language': None, 'base...",* Petrobras slumps after its CEO to step down ...
3,* Petrobras slumps after its CEO to step down...,"{'type': 'text/html', 'language': None, 'base'...",c141dfc5-ea0c-3778-843f-b503bd4966ba,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Wed, 15 May 2024 15:07:14 +0000","(2024, 5, 15, 15, 7, 14, 2, 136, 0)",EMERGING MARKETS-Most Latam FX up on softer U....,"{'type': 'text/plain', 'language': None, 'base...",* Petrobras slumps after its CEO to step down ...
4,* Petrobras shares fall after fuel sales hurt...,"{'type': 'text/html', 'language': None, 'base'...",90b5475c-6953-384e-8d8c-0eb024d77f3c,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Tue, 14 May 2024 20:06:12 +0000","(2024, 5, 14, 20, 6, 12, 1, 135, 0)","EMERGING MARKETS-Colombia, Chile outperform br...","{'type': 'text/plain', 'language': None, 'base...",* Petrobras shares fall after fuel sales hurt ...
5,* Brazil's Hapvida jumps after higher Q1 prof...,"{'type': 'text/html', 'language': None, 'base'...",7715d46c-04c7-314c-af28-df2946fd7354,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Tue, 14 May 2024 15:08:09 +0000","(2024, 5, 14, 15, 8, 9, 1, 135, 0)",EMERGING MARKETS-Most Latam FX inch up in lead...,"{'type': 'text/plain', 'language': None, 'base...",* Brazil's Hapvida jumps after higher Q1 profi...
6,* Banxico chief says rate cut a possibility a...,"{'type': 'text/html', 'language': None, 'base'...",897babf7-12ec-3526-a95a-f07d28cd9f40,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Mon, 13 May 2024 19:45:43 +0000","(2024, 5, 13, 19, 45, 43, 0, 134, 0)","EMERGING MARKETS-Latam FX, stocks flat as inve...","{'type': 'text/plain', 'language': None, 'base...",* Banxico chief says rate cut a possibility at...


In [6]:
from datetime import datetime

def convertir_fecha(fecha):
        fecha_obj = datetime.strptime(fecha, "%a, %d %b %Y %H:%M:%S %z")
        return fecha_obj.strftime("%d/%m/%Y")

df["Date"] = df["published"].apply(convertir_fecha)

In [7]:
df

,summary,summary_detail,id,guidislink,links,link,published,published_parsed,title,title_detail,contenido,Date
0,While the is up nearly 4% in 2025 and isn’t f...,"{'type': 'text/html', 'language': None, 'base'...",b5625d64-fbed-3b0f-a232-17e0fc421b37,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/m/b5625d64-fbed-3b0f...,"Thu, 20 Feb 2025 19:57:00 +0000","(2025, 2, 20, 19, 57, 0, 3, 51, 0)",The S&P 500 Could Keep Climbing. These Global ...,"{'type': 'text/plain', 'language': None, 'base...",While the is up nearly 4% in 2025 and isn’t fa...,20/02/2025
1,* Chile's peso at four month highs * Rising...,"{'type': 'text/html', 'language': None, 'base'...",1905da4a-7bc5-3840-b797-7383b6e3f1f2,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Thu, 16 May 2024 14:55:41 +0000","(2024, 5, 16, 14, 55, 41, 3, 137, 0)",EMERGING MARKETS-Most Latam FX fall as dollar ...,"{'type': 'text/plain', 'language': None, 'base...",* Chile's peso at four month highs * Rising ir...,16/05/2024
2,* Petrobras slumps after its CEO to step down...,"{'type': 'text/html', 'language': None, 'base'...",887dc1b6-1ece-3d6b-bb46-e0b220e3488a,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Wed, 15 May 2024 19:40:42 +0000","(2024, 5, 15, 19, 40, 42, 2, 136, 0)","EMERGING MARKETS-Mexico, Chile lead Latam FX u...","{'type': 'text/plain', 'language': None, 'base...",* Petrobras slumps after its CEO to step down ...,15/05/2024
3,* Petrobras slumps after its CEO to step down...,"{'type': 'text/html', 'language': None, 'base'...",c141dfc5-ea0c-3778-843f-b503bd4966ba,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Wed, 15 May 2024 15:07:14 +0000","(2024, 5, 15, 15, 7, 14, 2, 136, 0)",EMERGING MARKETS-Most Latam FX up on softer U....,"{'type': 'text/plain', 'language': None, 'base...",* Petrobras slumps after its CEO to step down ...,15/05/2024
4,* Petrobras shares fall after fuel sales hurt...,"{'type': 'text/html', 'language': None, 'base'...",90b5475c-6953-384e-8d8c-0eb024d77f3c,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Tue, 14 May 2024 20:06:12 +0000","(2024, 5, 14, 20, 6, 12, 1, 135, 0)","EMERGING MARKETS-Colombia, Chile outperform br...","{'type': 'text/plain', 'language': None, 'base...",* Petrobras shares fall after fuel sales hurt ...,14/05/2024
5,* Brazil's Hapvida jumps after higher Q1 prof...,"{'type': 'text/html', 'language': None, 'base'...",7715d46c-04c7-314c-af28-df2946fd7354,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Tue, 14 May 2024 15:08:09 +0000","(2024, 5, 14, 15, 8, 9, 1, 135, 0)",EMERGING MARKETS-Most Latam FX inch up in lead...,"{'type': 'text/plain', 'language': None, 'base...",* Brazil's Hapvida jumps after higher Q1 profi...,14/05/2024
6,* Banxico chief says rate cut a possibility a...,"{'type': 'text/html', 'language': None, 'base'...",897babf7-12ec-3526-a95a-f07d28cd9f40,False,"[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://finance.yahoo.com/news/emerging-market...,"Mon, 13 May 2024 19:45:43 +0000","(2024, 5, 13, 19, 45, 43, 0, 134, 0)","EMERGING MARKETS-Latam FX, stocks flat as inve...","{'type': 'text/plain', 'language': None, 'base...",* Banxico chief says rate cut a possibility at...,13/05/2024


In [10]:
df.columns.tolist()


['summary',
 'summary_detail',
 'id',
 'guidislink',
 'links',
 'link',
 'published',
 'published_parsed',
 'title',
 'title_detail',
 'contenido',
 'Date']

In [11]:
df = df.drop(['summary_detail', 'id', 'guidislink', 'links', 'published','published_parsed','title_detail'], axis=1)


In [12]:
df

,summary,link,title,contenido,Date
0,While the is up nearly 4% in 2025 and isn’t f...,https://finance.yahoo.com/m/b5625d64-fbed-3b0f...,The S&P 500 Could Keep Climbing. These Global ...,While the is up nearly 4% in 2025 and isn’t fa...,20/02/2025
1,* Chile's peso at four month highs * Rising...,https://finance.yahoo.com/news/emerging-market...,EMERGING MARKETS-Most Latam FX fall as dollar ...,* Chile's peso at four month highs * Rising ir...,16/05/2024
2,* Petrobras slumps after its CEO to step down...,https://finance.yahoo.com/news/emerging-market...,"EMERGING MARKETS-Mexico, Chile lead Latam FX u...",* Petrobras slumps after its CEO to step down ...,15/05/2024
3,* Petrobras slumps after its CEO to step down...,https://finance.yahoo.com/news/emerging-market...,EMERGING MARKETS-Most Latam FX up on softer U....,* Petrobras slumps after its CEO to step down ...,15/05/2024
4,* Petrobras shares fall after fuel sales hurt...,https://finance.yahoo.com/news/emerging-market...,"EMERGING MARKETS-Colombia, Chile outperform br...",* Petrobras shares fall after fuel sales hurt ...,14/05/2024
5,* Brazil's Hapvida jumps after higher Q1 prof...,https://finance.yahoo.com/news/emerging-market...,EMERGING MARKETS-Most Latam FX inch up in lead...,* Brazil's Hapvida jumps after higher Q1 profi...,14/05/2024
6,* Banxico chief says rate cut a possibility a...,https://finance.yahoo.com/news/emerging-market...,"EMERGING MARKETS-Latam FX, stocks flat as inve...",* Banxico chief says rate cut a possibility at...,13/05/2024


In [13]:
df.to_csv('../news/noticias_yahoo.csv', index=False, encoding='utf-8')